## 0. Import Library

In [2]:
from typing import List

import gzip
import os
import pickle
import random
import pandas as pd
import sklearn
from tqdm import tqdm

import numpy as np
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

/opt/anaconda3/envs/chatbot/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
def set_seed(self, seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(42)

In [4]:
print(pd.__version__)
print(np.__version__)
print(sklearn.__version__)
print(pickle.format_version)

1.5.2
1.24.1
1.2.0
4.0


## 1. load dataset

In [5]:
import gzip
import pickle

train_path = "../data/train.pickle"
valid_path = "../data/validation.pickle"

with gzip.open(train_path, 'rb') as f:
    train_data = pickle.load(f)

with gzip.open(valid_path, 'rb') as f:
    valid_data = pickle.load(f)

In [6]:
len(train_data), len(valid_data)

(92523, 10700)

## 2. Tokenizing datasets

- TF-IDF 계산해서 문서별 top3 키워드 구하기

In [7]:
from konlpy.tag import Mecab

mecab = Mecab()

In [8]:
summaries = [ann['summary1'] for ann in valid_data['annotation']]
summaries[:5]

['강 의원은 민간자격증 논란과 관련해 교육부 차원에서 민간자격증 제도에 대한 검정이 필요하다고 했다. ',
 '교육과학기술부장관 이 씨는 위탁업무이기 때문이라고 말하자 김 위원은 위탁업무여도 어느 정도 사안은 보고 받아야 하지 않냐고 했다.',
 '박 위원은 교육부를 해체시켜 연구학습부로 개편하자고 한 장관이 교육 자율성에 대한 강한 신념을 갖고 있는 것으로 이해했다.',
 '컨트롤타워가 누군지 묻는 이 위원의 말에 이 장관은 국과위를 통해 컨트롤타워 기능이 시현된다고 보면 될 것이라고 답했다.',
 '이 위원은 이 정부의 최대 국정 파탄의 예는 과학기술에 대한 부분으로 그 원인의 제공이 전담 부처나 컨트롤타워가 혼재되어 있다 하자 교육과학기술부장관 이 씨는 그렇지 않다 했다.']

In [9]:
print(summaries[0])
print(mecab.nouns(summaries[0]))

강 의원은 민간자격증 논란과 관련해 교육부 차원에서 민간자격증 제도에 대한 검정이 필요하다고 했다. 
['강', '의원', '민간', '자격증', '논란', '관련', '교육부', '차원', '민간', '자격증', '제도', '검정', '필요']


### 1. 경우의 수 정의

#### 1. unigram, 명사만 추출할 경우

In [10]:
vectorizer = CountVectorizer(ngram_range=(1,1), tokenizer=lambda x: mecab.nouns(x))  # vectorizer.vocabulary_에 token과 index 정보 저장
X = vectorizer.fit_transform(summaries)  # tf matrix 계산

transformer = TfidfTransformer()
X = transformer.fit_transform(X)
print(X.shape)

idx = np.argsort(X.toarray(), axis=1)
idx2vocab = [vocab for vocab, idx in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]
print(idx.shape)

/Users/daniel/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(10700, 14730)
(10700, 14730)


In [11]:
topk = 3

for i in range(5):
    lst = []
    for j in range(topk):
        id_ = idx[i][::-1][j]  # np.argsort()의 내림차순 정렬
        lst.append(idx2vocab[id_])
    print(f"{i}번째 문서 : {summaries[i]}")
    print(f"{i}번째 문서의 Top3 단어 : {lst}\n")

0번째 문서 : 강 의원은 민간자격증 논란과 관련해 교육부 차원에서 민간자격증 제도에 대한 검정이 필요하다고 했다. 
0번째 문서의 Top3 단어 : ['자격증', '민간', '검정']

1번째 문서 : 교육과학기술부장관 이 씨는 위탁업무이기 때문이라고 말하자 김 위원은 위탁업무여도 어느 정도 사안은 보고 받아야 하지 않냐고 했다.
1번째 문서의 Top3 단어 : ['위탁', '업무', '술부']

2번째 문서 : 박 위원은 교육부를 해체시켜 연구학습부로 개편하자고 한 장관이 교육 자율성에 대한 강한 신념을 갖고 있는 것으로 이해했다.
2번째 문서의 Top3 단어 : ['신념', '해체', '학습']

3번째 문서 : 컨트롤타워가 누군지 묻는 이 위원의 말에 이 장관은 국과위를 통해 컨트롤타워 기능이 시현된다고 보면 될 것이라고 답했다.
3번째 문서의 Top3 단어 : ['타워', '컨트롤', '누군지']

4번째 문서 : 이 위원은 이 정부의 최대 국정 파탄의 예는 과학기술에 대한 부분으로 그 원인의 제공이 전담 부처나 컨트롤타워가 혼재되어 있다 하자 교육과학기술부장관 이 씨는 그렇지 않다 했다.
4번째 문서의 Top3 단어 : ['파탄', '혼재', '타워']



#### 2. unigram, 형태소 추출할 경우

In [12]:
vectorizer = CountVectorizer(ngram_range=(1,1), tokenizer=lambda x: mecab.morphs(x))  # vectorizer.vocabulary_에 token과 index 정보 저장
X = vectorizer.fit_transform(summaries)  # tf matrix 계산

transformer = TfidfTransformer()
X = transformer.fit_transform(X)
print(X.shape)

idx = np.argsort(X.toarray(), axis=1)
idx2vocab = [vocab for vocab, idx in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]
print(idx.shape)

(10700, 18985)
(10700, 18985)


In [13]:
topk = 3

for i in range(5):
    lst = []
    for j in range(topk):
        id_ = idx[i][::-1][j]  # np.argsort()의 내림차순 정렬
        lst.append(idx2vocab[id_])
    print(f"{i}번째 문서 : {summaries[i]}")
    print(f"{i}번째 문서의 Top3 단어 : {lst}\n")

0번째 문서 : 강 의원은 민간자격증 논란과 관련해 교육부 차원에서 민간자격증 제도에 대한 검정이 필요하다고 했다. 
0번째 문서의 Top3 단어 : ['자격증', '민간', '검정']

1번째 문서 : 교육과학기술부장관 이 씨는 위탁업무이기 때문이라고 말하자 김 위원은 위탁업무여도 어느 정도 사안은 보고 받아야 하지 않냐고 했다.
1번째 문서의 Top3 단어 : ['위탁', '업무', '여도']

2번째 문서 : 박 위원은 교육부를 해체시켜 연구학습부로 개편하자고 한 장관이 교육 자율성에 대한 강한 신념을 갖고 있는 것으로 이해했다.
2번째 문서의 Top3 단어 : ['신념', '해체', '강한']

3번째 문서 : 컨트롤타워가 누군지 묻는 이 위원의 말에 이 장관은 국과위를 통해 컨트롤타워 기능이 시현된다고 보면 될 것이라고 답했다.
3번째 문서의 Top3 단어 : ['타워', '컨트롤', '누군지']

4번째 문서 : 이 위원은 이 정부의 최대 국정 파탄의 예는 과학기술에 대한 부분으로 그 원인의 제공이 전담 부처나 컨트롤타워가 혼재되어 있다 하자 교육과학기술부장관 이 씨는 그렇지 않다 했다.
4번째 문서의 Top3 단어 : ['혼재', '파탄', '타워']



#### 3. bigram, 명사만 추출할 경우

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer(ngram_range=(1,2), tokenizer=lambda x: mecab.nouns(x))  # vectorizer.vocabulary_에 token과 index 정보 저장
X = vectorizer.fit_transform(summaries)  # tf matrix 계산

transformer = TfidfTransformer()
X = transformer.fit_transform(X)
print(X.shape)

idx = np.argsort(X.toarray(), axis=1)
idx2vocab = [vocab for vocab, idx in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]
print(idx.shape)

(10700, 123529)
(10700, 123529)


In [15]:
topk = 3

for i in range(5):
    lst = []
    for j in range(topk):
        id_ = idx[i][::-1][j]  # np.argsort()의 내림차순 정렬
        lst.append(idx2vocab[id_])
    print(f"{i}번째 문서 : {summaries[i]}")
    print(f"{i}번째 문서의 Top3 단어 : {lst}\n")

0번째 문서 : 강 의원은 민간자격증 논란과 관련해 교육부 차원에서 민간자격증 제도에 대한 검정이 필요하다고 했다. 
0번째 문서의 Top3 단어 : ['민간 자격증', '자격증', '민간']

1번째 문서 : 교육과학기술부장관 이 씨는 위탁업무이기 때문이라고 말하자 김 위원은 위탁업무여도 어느 정도 사안은 보고 받아야 하지 않냐고 했다.
1번째 문서의 Top3 단어 : ['위탁 업무', '위탁', '업무']

2번째 문서 : 박 위원은 교육부를 해체시켜 연구학습부로 개편하자고 한 장관이 교육 자율성에 대한 강한 신념을 갖고 있는 것으로 이해했다.
2번째 문서의 Top3 단어 : ['교육부 해체', '개편 장관', '자율 신념']

3번째 문서 : 컨트롤타워가 누군지 묻는 이 위원의 말에 이 장관은 국과위를 통해 컨트롤타워 기능이 시현된다고 보면 될 것이라고 답했다.
3번째 문서의 Top3 단어 : ['컨트롤 타워', '타워', '컨트롤']

4번째 문서 : 이 위원은 이 정부의 최대 국정 파탄의 예는 과학기술에 대한 부분으로 그 원인의 제공이 전담 부처나 컨트롤타워가 혼재되어 있다 하자 교육과학기술부장관 이 씨는 그렇지 않다 했다.
4번째 문서의 Top3 단어 : ['타워 혼재', '혼재 교육', '제공 전담']



#### 4. 복합 명사로 합쳐서 vocab 구성할 경우

In [16]:
def add_compound_noun(phrase):
    """Mecab으로 형태소 분석 후, 명사(일반 명사(NNG), 고유 명사(NNP))가 연속으로 나올 때 복합 명사로 합쳐서 반환"""
    result = []
    pos_result = mecab.pos(phrase)
    idx = 0
    comp_word = ""

    while idx < len(pos_result):
        word, pos = pos_result[idx]
        if pos in ["NNG", "NNP"]:
            if comp_word != "":
                comp_word = f"{comp_word} {word}"  # 띄어쓰기로 구분하여 명사 합성
            else:
                comp_word += word
        else:
            if comp_word != "":
                result.append(comp_word)
            comp_word = ""  # 일반 명사, 고유 명사가 아닌 토큰이 나오면 다시 초기화
        idx += 1
    return result

In [17]:
def extract_compound_noun(phrase):
    """Mecab으로 형태소 분석 후, 명사(일반 명사(NNG), 고유 명사(NNP))가 연속으로 나올 때 복합 명사로 합쳐서 반환"""
    result = []
    pos_result = mecab.pos(phrase)
    idx = 0
    comp_word = ""

    while idx < len(pos_result):
        word, pos = pos_result[idx]
        if pos in ["NNG", "NNP"]:
            if comp_word != "":
                comp_word = f"{comp_word} {word}"  # 띄어쓰기로 구분하여 명사 합성
            else:
                comp_word += word
        else:
            if len(comp_word.split(' ')) > 1:
                result.append(comp_word)
            comp_word = ""  # 일반 명사, 고유 명사가 아닌 토큰이 나오면 다시 초기화
        idx += 1
    return result

In [18]:
# Test : 형태소 분석 후 "명사"로 추출된 토큰들은 모두 띄어쓰기로 나눠져 있지 않음을 확인
for i in summaries:
    temp = mecab.nouns(i)
    for j in temp:
        if len(j.split(" ")) > 1:
            print(i, j)

In [19]:
add_compound_noun(summaries[0])

['강 의원', '민간 자격증 논란', '관련', '교육부 차원', '민간 자격증 제도', '검정', '필요']

### 2. train data

In [20]:
train_data.head()

,source,ids,passage,annotation
0,report,REPORT-minute-00001-00004,"위원장 신학용] ""수고하셨습니다. 다음은 존경하는 현영희 위원님 인사말씀해 주세...",{'summary1': '아이들의 미래를 열어가는 장을 만들 수 있는 교과위에 오기...
1,report,REPORT-minute-00001-00006,"위원장 신학용] ""수고하셨고요. 이상민 위원님 인사말씀해 주세요.""\n이상민 위원...",{'summary1': '이 위원은 교육과 과학이 국가의 미래이고 서민들의 삶을 결...
2,report,REPORT-minute-00001-00019,"교육과학기술부장관 이주호] ""96년부터 시행을 하고 있습니다.""\n강은희 위원] ""...",{'summary1': '강 위원은 진보 성향의 교육감을 둔 지역은 공교롭게도 20...
3,report,REPORT-minute-00001-00032,"교육과학기술부장관 이주호] ""저희도 그 부분이 굉장히 가장 중요한 부분이라고 생각하...",{'summary1': '이 교육과학기술부장관은 주 5일제가 잘 정착하기 위해 학교...
4,report,REPORT-minute-00001-00033,"교육과학기술부장관 이주호] ""예.""\n김상희 위원] ""결정을 해서 지금 이제 권고를...",{'summary1': '김 위원은 야당 국회의원인 시인 도 씨의 작품을 교과서에서...


In [21]:
summaries = [ann['summary1'] for ann in train_data['annotation']]
summaries[:5]

['아이들의 미래를 열어가는 장을 만들 수 있는 교과위에 오기를 기대했던 박 위원은 아이들을 위한 일에 힘을 보태고 싶다고 말했다.',
 '이 위원은 교육과 과학이 국가의 미래이고 서민들의 삶을 결정짓는 중요한 국가적 의제이므로 여야 가리지 않고 위원들과 협조해 성과를 창출하겠다고 했다.',
 '강 위원은 진보 성향의 교육감을 둔 지역은 공교롭게도 2012년 시ㆍ도 교육청 평가에서 매우 미흡 판정을 받았다고 했다.',
 '이 교육과학기술부장관은 주 5일제가 잘 정착하기 위해 학교 차원의 배려가 중요하다고 했다.',
 '김 위원은 야당 국회의원인 시인 도 씨의 작품을 교과서에서 삭제하는 것은 정치적으로 굉장히 민감해지는 사안이라며 장관이 이런 결정에 대한 보고를 못 받을 리 없다고 했다.']

In [22]:
print(summaries[0])
print(mecab.nouns(summaries[0]))

아이들의 미래를 열어가는 장을 만들 수 있는 교과위에 오기를 기대했던 박 위원은 아이들을 위한 일에 힘을 보태고 싶다고 말했다.
['아이', '미래', '장', '수', '교과', '위', '기대', '박', '위원', '아이', '일', '힘', '말']


In [23]:
# Test : 형태소 분석 후 "명사"로 추출된 토큰들은 모두 띄어쓰기로 나눠져 있지 않음을 확인 (1개 제외)
for a, b in enumerate(summaries):
    temp = mecab.nouns(b)
    for j in temp:
        if len(j.split(" ")) > 1:
            print(a, j)

69639 에듀 
69639 에듀 


In [24]:
# "에듀" -> "에듀 "로 나눠짐
mecab.nouns(train_data.iloc[69639, 3]['summary1'])

['에듀 ', '파인', '우려', '제기', '년', '월', '유치원', '도입', '에듀 ', '파인', '사용']

In [25]:
# Test : summaries 값 중 빈 값이 있는지 확인
for t in summaries:
    if len(t) == 0:
        print(t)

In [26]:
# vectorizer = CountVectorizer(tokenizer=lambda x: add_compound_noun(x))  # 복합명사 + 명사 : shape : (92523, 169582)
vectorizer = CountVectorizer(tokenizer=lambda x: extract_compound_noun(x))  # 복합명사만 : shape : (92523, 144243)
# vectorizer = CountVectorizer(tokenizer=lambda x: mecab.nouns(x))  # 명사 : shape : (92523, 36545)
X = vectorizer.fit_transform(summaries)  # tf matrix 계산

transformer = TfidfTransformer()
X = transformer.fit_transform(X)  # tf-idf 계산
print(X.shape)

idx = np.argsort(X.toarray(), axis=1)
print(idx.shape)
idx2vocab = [vocab for vocab, idx in tqdm(sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1]))]

/Users/daniel/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(92523, 144243)
(92523, 144243)


100%|██████████| 144243/144243 [00:00<00:00, 1511145.00it/s]


In [27]:
topk = 3
queries = []

for i in range(len(summaries)):
    lst = []
    for j in range(topk):
        id_ = idx[i][::-1][j]  # np.argsort()한 결과에 대해 내림차순 정렬
        lst.append(idx2vocab[id_])
    queries.append(lst)

In [28]:
train_data['query'] = queries
train_data.head()

,source,ids,passage,annotation,query
0,report,REPORT-minute-00001-00004,"위원장 신학용] ""수고하셨습니다. 다음은 존경하는 현영희 위원님 인사말씀해 주세...",{'summary1': '아이들의 미래를 열어가는 장을 만들 수 있는 교과위에 오기...,"[교과 위, 박 위원, 반탁 운동]"
1,report,REPORT-minute-00001-00006,"위원장 신학용] ""수고하셨고요. 이상민 위원님 인사말씀해 주세요.""\n이상민 위원...",{'summary1': '이 위원은 교육과 과학이 국가의 미래이고 서민들의 삶을 결...,"[힙합 특강, 반쪽 국회 개원, 반출 업무 훈령]"
2,report,REPORT-minute-00001-00019,"교육과학기술부장관 이주호] ""96년부터 시행을 하고 있습니다.""\n강은희 위원] ""...",{'summary1': '강 위원은 진보 성향의 교육감을 둔 지역은 공교롭게도 20...,"[미흡 판정, 교육청 평가, 진보 성향]"
3,report,REPORT-minute-00001-00032,"교육과학기술부장관 이주호] ""저희도 그 부분이 굉장히 가장 중요한 부분이라고 생각하...",{'summary1': '이 교육과학기술부장관은 주 5일제가 잘 정착하기 위해 학교...,"[학교 차원, 학기 술부 장관, 힙합 특강]"
4,report,REPORT-minute-00001-00033,"교육과학기술부장관 이주호] ""예.""\n김상희 위원] ""결정을 해서 지금 이제 권고를...",{'summary1': '김 위원은 야당 국회의원인 시인 도 씨의 작품을 교과서에서...,"[시인 도, 야당 국회의원, 김 위원]"


In [29]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92523 entries, 0 to 92522
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   source      92523 non-null  object
 1   ids         92523 non-null  object
 2   passage     92523 non-null  object
 3   annotation  92523 non-null  object
 4   query       92523 non-null  object
dtypes: object(5)
memory usage: 3.5+ MB


In [30]:
# Test : 3개의 Query Keyword로 구성되어 있지 않은 게 있는지 확인
for a, b in enumerate(train_data['query']):
    if len(b) != 3:
        print(a, b)

In [31]:
for i in summaries[:10]:
    print(i)
    print(extract_compound_noun(i), '\n')

아이들의 미래를 열어가는 장을 만들 수 있는 교과위에 오기를 기대했던 박 위원은 아이들을 위한 일에 힘을 보태고 싶다고 말했다.
['교과 위', '박 위원'] 

이 위원은 교육과 과학이 국가의 미래이고 서민들의 삶을 결정짓는 중요한 국가적 의제이므로 여야 가리지 않고 위원들과 협조해 성과를 창출하겠다고 했다.
[] 

강 위원은 진보 성향의 교육감을 둔 지역은 공교롭게도 2012년 시ㆍ도 교육청 평가에서 매우 미흡 판정을 받았다고 했다.
['강 위원', '진보 성향', '교육청 평가', '미흡 판정'] 

이 교육과학기술부장관은 주 5일제가 잘 정착하기 위해 학교 차원의 배려가 중요하다고 했다.
['학기 술부 장관', '학교 차원'] 

김 위원은 야당 국회의원인 시인 도 씨의 작품을 교과서에서 삭제하는 것은 정치적으로 굉장히 민감해지는 사안이라며 장관이 이런 결정에 대한 보고를 못 받을 리 없다고 했다.
['김 위원', '야당 국회의원', '시인 도'] 

김 위원은 성 평가원장을 사퇴 조치해 달라고 했고 교육과학기술부장관 이 씨는 지휘 감독의 소홀한 부분에 대해 유감을 표명했다고 말했다.
['김 위원', '성 평가 원장', '사퇴 조치', '학기 술부 장관', '지휘 감독'] 

교육과학기술부장관 이 씨는 교과서가 정치적 중립성이 최대한 보장되어야 하므로 검정을 하는 경우에도 위탁기관이 검정을 하게 되어 있다고 말했다.
['학기 술부 장관', '최대한 보장', '위탁 기관'] 

박 위원은 대구에서 10명의 중학생들이 투신자살을 했음에도 대구교육청은 우수평가를 받아 많은 지원을 받았다며 교육청이 잘 하고 있는 것이냐고 했다.
['박 위원', '대구 교육청', '우수 평가'] 

이 장관이 선관위에 위탁하게 된 과정에 대해 아는 것이 없어 위원들이 질의를 해도 답변을 못한다. 
[] 

권고 공문으로 출판사에 간 것은 합격 딱지가 붙어 있으며 수정된 것을 판단하는 것은 나중 문제이다.
['권고 공문', '합격 딱지', '나중 문제'] 



In [32]:
null_lst = [
    (a, b)
    for a, b in enumerate(summaries)
    if len(extract_compound_noun(b)) == 0
]
len(null_lst)

3653

In [33]:
with open('train_query.pickle', 'wb') as fw:
    pickle.dump(train_data, fw, protocol=pickle.HIGHEST_PROTOCOL)

### 3. validation_data

In [34]:
valid_data.head()

,source,ids,passage,annotation
0,report,REPORT-minute-00001-00021,"강은희 위원] ""마지막으로 한 가지 더 여쭙겠습니다. 최근 민간자격증 관해서도 ...",{'summary1': '강 의원은 민간자격증 논란과 관련해 교육부 차원에서 민간자...
1,report,REPORT-minute-00001-00034,"교육과학기술부장관 이주호] ""제가 앞에서도 말씀드렸지만 그게 위탁업무이기 때문에 그...",{'summary1': '교육과학기술부장관 이 씨는 위탁업무이기 때문이라고 말하자 ...
2,report,REPORT-minute-00001-00045,"교육과학기술부장관 이주호] ""예.""\n박혜자 위원] ""그 이유는 지역과 학교의 다양...",{'summary1': '박 위원은 교육부를 해체시켜 연구학습부로 개편하자고 한 장...
3,report,REPORT-minute-00001-00059,"교육과학기술부장관 이주호] ""예 어떻든 하여튼 최선 다해서……""\n이상민 위원] ""...",{'summary1': '컨트롤타워가 누군지 묻는 이 위원의 말에 이 장관은 국과위...
4,report,REPORT-minute-00001-00060,"이상민 위원] ""아니 보세요. 장관님 예산에 대한 조정ㆍ배분권만 갖고 있으면 대한민...",{'summary1': '이 위원은 이 정부의 최대 국정 파탄의 예는 과학기술에 대...


In [35]:
summaries = [ann['summary1'] for ann in valid_data['annotation']]
summaries[:5]

['강 의원은 민간자격증 논란과 관련해 교육부 차원에서 민간자격증 제도에 대한 검정이 필요하다고 했다. ',
 '교육과학기술부장관 이 씨는 위탁업무이기 때문이라고 말하자 김 위원은 위탁업무여도 어느 정도 사안은 보고 받아야 하지 않냐고 했다.',
 '박 위원은 교육부를 해체시켜 연구학습부로 개편하자고 한 장관이 교육 자율성에 대한 강한 신념을 갖고 있는 것으로 이해했다.',
 '컨트롤타워가 누군지 묻는 이 위원의 말에 이 장관은 국과위를 통해 컨트롤타워 기능이 시현된다고 보면 될 것이라고 답했다.',
 '이 위원은 이 정부의 최대 국정 파탄의 예는 과학기술에 대한 부분으로 그 원인의 제공이 전담 부처나 컨트롤타워가 혼재되어 있다 하자 교육과학기술부장관 이 씨는 그렇지 않다 했다.']

In [36]:
# Test : summaries 값 중 빈 값이 있는지 확인
for t in summaries:
    if len(t) == 0:
        print(t)

In [37]:
# Test : 형태소 분석 후 명사로 추출된 토큰들은 1개를 제외하고 모두 띄어쓰기로 나눠져 있지 않음을 확인
for a, b in enumerate(summaries):
    temp = mecab.nouns(b)
    for j in temp:
        if len(j.split(" ")) > 1:
            print(a, j)

In [38]:
# vectorizer = CountVectorizer(tokenizer=lambda x: add_compound_noun(x))  # 복합명사 + 명사 : shape : (92523, 169582)
vectorizer = CountVectorizer(tokenizer=lambda x: extract_compound_noun(x))  # 복합명사만 : shape : (92523, 144243)
# vectorizer = CountVectorizer(tokenizer=lambda x: mecab.nouns(x))  # 명사 : shape : (92523, 36545)
X = vectorizer.fit_transform(summaries)  # tf matrix 계산

transformer = TfidfTransformer()
X = transformer.fit_transform(X)  # tf-idf 계산
print(X.shape)

idx = np.argsort(X.toarray(), axis=1)
print(idx.shape)
idx2vocab = [vocab for vocab, idx in tqdm(sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1]))]

/Users/daniel/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(10700, 22781)
(10700, 22781)


100%|██████████| 22781/22781 [00:00<00:00, 1932732.70it/s]


In [39]:
topk = 3
queries = []

for i in range(len(summaries)):
    lst = []
    for j in range(topk):
        id_ = idx[i][::-1][j]  # np.argsort()한 결과에 대해 내림차순 정렬
        lst.append(idx2vocab[id_])
    queries.append(lst)

In [40]:
valid_data['query'] = queries
valid_data.head()

,source,ids,passage,annotation,query
0,report,REPORT-minute-00001-00021,"강은희 위원] ""마지막으로 한 가지 더 여쭙겠습니다. 최근 민간자격증 관해서도 ...",{'summary1': '강 의원은 민간자격증 논란과 관련해 교육부 차원에서 민간자...,"[민간 자격증 논란, 민간 자격증 제도, 교육부 차원]"
1,report,REPORT-minute-00001-00034,"교육과학기술부장관 이주호] ""제가 앞에서도 말씀드렸지만 그게 위탁업무이기 때문에 그...",{'summary1': '교육과학기술부장관 이 씨는 위탁업무이기 때문이라고 말하자 ...,"[위탁 업무, 정도 사안, 학기 술부 장관]"
2,report,REPORT-minute-00001-00045,"교육과학기술부장관 이주호] ""예.""\n박혜자 위원] ""그 이유는 지역과 학교의 다양...",{'summary1': '박 위원은 교육부를 해체시켜 연구학습부로 개편하자고 한 장...,"[연구 학습 부, 교육 자율, 박 위원]"
3,report,REPORT-minute-00001-00059,"교육과학기술부장관 이주호] ""예 어떻든 하여튼 최선 다해서……""\n이상민 위원] ""...",{'summary1': '컨트롤타워가 누군지 묻는 이 위원의 말에 이 장관은 국과위...,"[컨트롤 타워 기능, 국과 위, 컨트롤 타워]"
4,report,REPORT-minute-00001-00060,"이상민 위원] ""아니 보세요. 장관님 예산에 대한 조정ㆍ배분권만 갖고 있으면 대한민...",{'summary1': '이 위원은 이 정부의 최대 국정 파탄의 예는 과학기술에 대...,"[최대 국정 파탄, 전담 부처, 과학 기술]"


In [41]:
valid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10700 entries, 0 to 10699
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   source      10700 non-null  object
 1   ids         10700 non-null  object
 2   passage     10700 non-null  object
 3   annotation  10700 non-null  object
 4   query       10700 non-null  object
dtypes: object(5)
memory usage: 418.1+ KB


In [42]:
valid_query_series = valid_data['query']
valid_query_series

0                [민간 자격증 논란, 민간 자격증 제도, 교육부 차원]
1                      [위탁 업무, 정도 사안, 학기 술부 장관]
2                        [연구 학습 부, 교육 자율, 박 위원]
3                     [컨트롤 타워 기능, 국과 위, 컨트롤 타워]
4                      [최대 국정 파탄, 전담 부처, 과학 기술]
                          ...                  
10695           [힙합 곡인 디지털 싱글, 박용진 의원, 반 부패 정책]
10696           [힙합 곡인 디지털 싱글, 박용진 의원, 반 부패 정책]
10697    [올림픽 양궁 선수 김제덕, 예능 프로그램, 힙합 곡인 디지털 싱글]
10698           [힙합 곡인 디지털 싱글, 박용진 의원, 반 부패 정책]
10699                   [아들 젠, 레이먼킴 가족, 해피 선데이]
Name: query, Length: 10700, dtype: object

In [43]:
# Test : 3개의 Query Keyword로 구성되어 있지 않은 게 있는지 확인
for a, b in enumerate(valid_query_series):
    if len(b) != 3:
        print(a, b)

In [44]:
with open('valid_query.pickle', 'wb') as fw:
    pickle.dump(valid_data, fw, protocol=pickle.HIGHEST_PROTOCOL)